In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def is_surname(x):
    w = morph.parse(x)[0]
    if 'Surn' in w.tag:
        return True
    else:
        return False

In [ ]:
def make_all(line):
    res = []
    wrds = line.split()
    for langtype in ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct']:
        newforms = []
        try:
            for w in wrds:
                w = w.strip()
                w = morph.parse(w)[0]
                inflected = w.inflect({langtype})
                if inflected:
                    w_newform = inflected.word
                    if len(w_newform) > 0:
                        newforms.append(w_newform)
            if len(newforms) > 0:
                new_phrase = ' '.join(newforms)
                res.append(new_phrase)
        except Exception as ex:
            print(ex)
    return list(set(res))

In [ ]:
import psycopg2
conn = psycopg2.connect(config.DB_PROD)
# conn = psycopg2.connect(config.DB_STAGING)
conn.autocommit = True
cur = conn.cursor()

In [ ]:
cur.execute("SELECT id, name FROM keywords WHERE forms = '{}'")
list_from_db = cur.fetchall()

In [ ]:
len(list_from_db)

In [ ]:
for kw in list_from_db[0:5000]:
    k_id = kw[0]
    k_line = kw[1]
    forms_arr = make_all(k_line)
    if len(forms_arr) > 0:
        forms = ','.join(forms_arr)
        forms = '{' + forms + '}'
        cur.execute("UPDATE keywords SET forms = %s WHERE id = %s", (forms, k_id))
    else:
        print(k_line, forms_arr)

In [ ]:
cur.execute("SELECT id, name FROM customer_keywords WHERE forms = '{}'")
list_from_db = cur.fetchall()
len(list_from_db)

In [ ]:
for kw in list_from_db[0:5000]:
    k_id = kw[0]
    k_line = kw[1]
    forms_arr = make_all(k_line)
    if len(forms_arr) > 0:
        forms = ','.join(forms_arr)
        forms = '{' + forms + '}'
        cur.execute("UPDATE customer_keywords SET forms = %s WHERE id = %s", (forms, k_id))
    else:
        print(k_line, forms_arr)

In [ ]:
cur.execute("SELECT id, blacklist FROM customer_keywords WHERE blacklist_forms = '{}'")
list_from_db = cur.fetchall()
need_process = []
for l in list_from_db:
    if len(l[1]) > 0:
        need_process.append(l)
len(need_process)

In [ ]:
for kw in need_process[0:5000]:
    k_id = kw[0]
    k_line = kw[1]
    forms_arr = []
    for l in k_line:
        forms_arr += make_all(l)
    if len(forms_arr) > 0:
        forms = ','.join(forms_arr)
        forms = '{' + forms + '}'
        cur.execute("UPDATE customer_keywords SET blacklist_forms = %s WHERE id = %s", (forms, k_id))

In [ ]:
cur.close()
conn.close()